# 042-02-GPT-Advanced techniques-solution

* Written by Alexandre Gazagnes
* Last update: 2024-02-01

## About 

Context : 

Let's get the party continue ! 

Data  : 

**You can find the dataset [here](https://gist.githubusercontent.com/AlexandreGazagnes/cabe445634a092d308d17a883a305a75/raw/9f785f0f02739ac6352e1d583323771d55270221/nlp.csv).**

## Preliminaries

### System

These commands will display the system information:

Uncomment theses lines if needed. 

In [ ]:
# pwd

In [ ]:
# cd ..

In [ ]:
# ls

Install various Librairies : 

In [ ]:
# !pip install -r requirements.txt >> pip.log
# !pip freeze >> pip.freeze

### Import 

In [ ]:
import os, sys, warnings, secrets, datetime
import pickle
from IPython.display import display

# from joblib import dump, load

from abc import ABC

import json
import logging

In [ ]:
from redlines import Redlines

In [ ]:
from dotenv import load_dotenv, find_dotenv

In [ ]:
import pandas as pd

# import numpy as np

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px

In [ ]:
import transformers

In [ ]:
import openai
from openai import OpenAI

# from load_dotenv import load_dotenv
from dotenv import dotenv_values

In [ ]:
import langchain

In [ ]:
import requests  # HTTP client

In [ ]:
def now():
    """ """
    date = datetime.datetime.now()
    date = str(date)
    date = date[:19]
    date = date.replace(" ", "_").replace(":", "-")

    return date

### Graphs and Settings

In [ ]:
# sns.set()

In [ ]:
warnings.filterwarnings("ignore")
# warnings.filterwarnings(action="once")

### Env management

In [ ]:
def manage_env(fn=".env/.env.prod"):
    """ """
    config = dotenv_values(fn)

    if not config:
        config = {
            # GENERIC
            "ENV": os.getenv("ENV", "LOCAL"),
            "LOG_LEVEL": os.getenv("LOG_LEVEL", "INFO"),
            "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY", None),
        }

    return dict(config)

### Data

url of the dataset :

In [ ]:
url = "https://gist.githubusercontent.com/AlexandreGazagnes/cabe445634a092d308d17a883a305a75/raw/d2014e8a34bba3c1be3ec8936bb338fb42888f24/nlp.csv"

Download the dataset : 

In [ ]:
df = pd.read_csv(url)
df.head(5)

Keep a copy of the df : 

In [ ]:
DF = df.copy()

## Simple Embedding

In [ ]:
client = openai.OpenAI()

If you do have troubles, please run the following : 

In [ ]:
# _ = load_dotenv(find_dotenv())

# openai.api_key = os.getenv("OPENAI_API_KEY")

Doc : 

In [ ]:
doc = df.description.iloc[0]
doc

Just a try : 

In [ ]:
response = client.embeddings.create(
    input=doc,
    model="text-embedding-3-small",
)

What is response : 

In [ ]:
response

The vector : 

In [ ]:
vector = response.data[0].embedding
vector

Size?

In [ ]:
len(vector)

## Guidelines for Prompting

### Basic

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo", prompt="Who won the world series in 2020?"
)
response

In [ ]:
response.choices[0].message.content

With Prompt Eng. : 

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {
            "role": "assistant",
            "content": "The Los Angeles Dodgers won the World Series in 2020.",
        },
        {"role": "user", "content": "Where was it played?"},
    ],
)
response

In [ ]:
response.choices[0]

In [ ]:
response.choices[0].message

In [ ]:
response.choices[0].message.content

### Delimiters

Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

Text to summarize : 

In [ ]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

print(text)

Prompt : 

In [ ]:
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

print(prompt)

Output : 

In [ ]:
print(prompt)

Helper function : 

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):

    messages = [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    return response.choices[0].message.content

Output : 

In [ ]:
response = get_completion(prompt)
print(response)

### Secific Format

Prompt : 

In [ ]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""

In [ ]:
response = get_completion(prompt)
print(response)

### Adding conditions

In [ ]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""

In [ ]:
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

In [ ]:
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

In [ ]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""

In [ ]:
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""

In [ ]:
response = get_completion(prompt)


print("Completion for Text 2:")
print(response)

### One / Few shot learning

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Today the weather is fantastic"},
        {"role": "assistant", "content": "positive"},
        {"role": "user", "content": "I don't like your attitude"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "That shot selection was awful"},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

In [ ]:
response = get_completion(prompt)
print(response)

### Multi step prompting

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""

In [ ]:
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

In [ ]:
response = get_completion(prompt_1)
response

In [ ]:
print("Completion for prompt 1:")
print(response)

### Using structured output

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""

In [ ]:
response = get_completion(prompt_2)

In [ ]:
print("\nCompletion for prompt 2:")
print(response)

### Futo feed back loops 

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

In [ ]:
response = get_completion(prompt)
print(response)

Not good ! 

In [ ]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

In [ ]:
response = get_completion(prompt)
print(response)

### Hallucinatuion

In [ ]:
prompt = f"""
Tell me about my 102 years old grand mother who crossed the pacific sea by swiming in 1 Day, wihout using her arms and her legs
"""

In [ ]:
response = get_completion(prompt)
print(response)

In [ ]:
response = get_completion(prompt, model="gpt-4")
print(response)

## Summarizing

Text to summarize


In [ ]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""


Summarize with a word/sentence/character limit


In [ ]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

In [ ]:
response = get_completion(prompt)
print(response)

Summarize with a focus on shipping and delivery


In [ ]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

In [ ]:
response = get_completion(prompt)
print(response)


Summarize with a focus on price and value


In [ ]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value. 

Review: ```{prod_review}```
"""

In [ ]:
response = get_completion(prompt)
print(response)

Summaries include topics that are not related to the topic of focus.



Try "extract" instead of "summarize"


In [ ]:
prompt = f"""
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

In [ ]:
response = get_completion(prompt)
print(response)


Summarize multiple product reviews


In [ ]:
review_1 = prod_review

In [ ]:
# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

In [ ]:
# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

In [ ]:
# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [ ]:
reviews = [review_1, review_2, review_3, review_4]

iterate for each product

In [ ]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.


 ### Basic

In [ ]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""

In [ ]:
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""

In [ ]:
response = get_completion(prompt)
print(response)

### Multiple 

In [ ]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```
"""

In [ ]:
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""

In [ ]:
response = get_completion(prompt)
print(response)

### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!



In [ ]:
user_messages = [
    "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal
    "Mi monitor tiene píxeles que no se iluminan.",  # My monitor has pixels that are not lighting
    "Il mio mouse non funziona",  # My mouse is not working
    "Mój klawisz Ctrl jest zepsuty",  # My keyboard has a broken control key
    "我的屏幕在闪烁",  # My screen is flashing
]

In [ ]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")


### Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.


In [ ]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""

In [ ]:
response = get_completion(prompt)
print(response)


### Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.


In [ ]:
data_json = {
    "resturant employees": [
        {"name": "Shyam", "email": "shyamjaiswal@gmail.com"},
        {"name": "Bob", "email": "bob32@gmail.com"},
        {"name": "Jai", "email": "jai87@gmail.com"},
    ]
}

In [ ]:
prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""

In [ ]:
response = get_completion(prompt)
print(response)

In [ ]:
from IPython.display import display, Markdown, Latex, HTML, JSON

display(HTML(response))

### Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.



In [ ]:
text = [
    "The girl with the black and white puppies have a ball.",  # The girl has a ball.
    "Yolanda has her notebook.",  # ok
    "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
    "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
    "Your going to need you’re notebook.",  # Homonyms
    "That medicine effects my ability to sleep. Have you heard of the butterfly affect?",  # Homonyms
    "This phrase is to cherck chatGPT for speling abilitty",  # spelling
]

In [ ]:
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

In [ ]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""

In [ ]:
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

In [ ]:
diff = Redlines(text, response)
display(Markdown(diff.output_markdown))

In [ ]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""

In [ ]:
response = get_completion(prompt)
display(Markdown(response))

## Inferring

In [ ]:
## Product review text

lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""


### Sentiment

In [ ]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
## Identify types of emotions

prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
## Identify anger

prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


### Extraction

In [ ]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""

In [ ]:
response = get_completion(prompt)
print(response)


### Multi Tasking 

In [ ]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


### Long Texts


In [ ]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

In [ ]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""

In [ ]:
response = get_completion(prompt)
print(response)

In [ ]:
response.split(sep=",")

In [ ]:
topic_list = [
    "nasa",
    "local government",
    "engineering",
    "employee satisfaction",
    "federal government",
]

### Alerting


Make a news alert for certain topics


In [ ]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""

In [ ]:
response = get_completion(prompt)
print(response)

In [ ]:
topic_dict = {i.split(": ")[0]: int(i.split(": ")[1]) for i in response.split(sep="\n")}
if topic_dict["nasa"] == 1:
    print("ALERT: New NASA story!")

## Advanced prompting system

### ABC

In [ ]:
class BasePrompt(ABC):
    """Basic prompt for the user.

    Methods:
        - dictize() : return dict of messages
        - jsonize() : return json of messages
        - strize() : return str of messages
    """

    prompt_type = "base"
    user_delimiter = "<!> BASE USER DELIMITER <!>"
    insight_delimiter = "<!> BASE INSIGHT DELIMITER <!>"
    insight = "<!> BASE INSIGHT <!>"
    system_message_template = """"<!> BASE SYSTEM MESSAGE TEMPLATE <!>"""
    user_message_template = """"<!> BASE USER MESSAGE TEMPLATE <!>"""

    def __init__(
        self,
        data: str,
        insight: str = None,
        system_message_template: str = None,
        user_message_template: str = None,
        user_delimiter: str = None,
        insight_delimiter: str = None,
        prompt_type: str = None,
    ) -> None:
        """Build a system message from a template and a data

        Positional arguments:
            - data : pd.DataFrame | str | dict : data to use

        Optional arguments:
            - system_message_template : str : system message template
            - user_delimiter : str : user_delimiter for user
            - user_message_template : str : user message

        Returns:
            - system_message : dict[str:str] : system message
        """

        # logging.info("InsightBasePrompt ==>  __init__ CALLED")

        # args

        self.data = data
        self.insight = insight if insight else self.insight

        self.system_message_template = (
            system_message_template
            if system_message_template
            else self.system_message_template
        )

        self.user_message_template = (
            user_message_template
            if user_message_template
            else self.user_message_template
        )

        self.user_delimiter = user_delimiter if user_delimiter else self.user_delimiter
        self.insight_delimiter = (
            insight_delimiter if insight_delimiter else self.insight_delimiter
        )

        self.prompt_type = prompt_type if prompt_type else self.prompt_type

    def _build_system_message(self):
        raise AttributeError("ABC _build_system_message => Sould not be called")

    def _build_user_message(self):
        raise AttributeError("ABC _build_user_message_template => Sould not be called")

    def _build_messages(self):
        """build messages"""

        # logging.info("ABC => _build_messages => CALLED")

        system_content = self._build_system_message()
        user_content = self._build_user_message()

        messages = [
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content},
        ]

        return messages

    def dictize(self) -> dict:
        """return dict of messages"""

        return self._build_messages()

    def jsonize(self) -> str:
        """ """

        raise NotImplementedError("not implemented yet")

    def strize(self) -> str:
        """ """

        messages_str = []

        for message in self._build_messages():
            tmp = f"{message['role']}:\n{message['content']}\n\n"
            messages_str.append(tmp)

        return "".join(messages_str)

    def __repr__(self) -> str:
        if self.prompt_type == "base":
            return "BasePrompt(ABC)"
        if self.prompt_type == "basic":
            return "BasicPrompt(BasicBasePrompt)"
        if self.prompt_type == "insight":
            return "InsightPrompt(InsightBasePrompt)"

### BasePromts

In [ ]:
class BasicBasePrompt(BasePrompt):
    """ """

    def __init__(
        self,
        data: str,
        system_message_template: str = None,
        user_message_template: str = None,
        user_delimiter: str = None,
    ) -> None:
        """ """

        # logging.info("BasicBasePrompt ==>  __init__ CALLED")

        super().__init__(
            data=data,
            system_message_template=system_message_template,
            user_message_template=user_message_template,
            user_delimiter=user_delimiter,
            prompt_type="basic",
        )

    def _build_system_message(self):
        """build system message from template and data"""

        # logging.info("BasicBasePrompt => _build_system_message => CALLED")

        system_message = self.system_message_template.replace(
            "--USER_DELIMITER--", self.user_delimiter
        )

        system_message = system_message.replace("\n\n", "\n").replace("\n\n", "\n")

        return system_message

    def _build_user_message(self):
        """_build_user_message"""

        # logging.info("BasicBasePrompt => _build_user_message => CALLED")

        user_message = self.user_message_template.replace(
            "--DATA--", str(self.data)
        ).replace("--USER_DELIMITER--", self.user_delimiter)

        return user_message

In [ ]:
class InsightBasePrompt(BasePrompt):
    """ """

    def __init__(
        self,
        data: str,
        insight: str,
        system_message_template: str = None,
        user_message_template: str = None,
        user_delimiter: str = None,
        insight_delimiter: str = None,
    ) -> None:
        """ """

        # logging.info("InsightBasePrompt ==>  __init__ CALLED")

        super().__init__(
            data=data,
            insight=insight,
            system_message_template=system_message_template,
            user_message_template=user_message_template,
            user_delimiter=user_delimiter,
            insight_delimiter=insight_delimiter,
            prompt_type="insight",
        )

    def _build_system_message(self):
        """build system message from template and data"""

        # logging.info("InsightBasePrompt => _build_system_message => CALLED")

        # logging.info(f"self.system_message_template : {self.system_message_template}")

        system_message = self.system_message_template.replace(
            "--USER_DELIMITER--", self.user_delimiter
        ).replace("--INSIGHT_DELIMITER--", self.insight_delimiter)

        system_message = system_message.replace("\n\n", "\n").replace("\n\n", "\n")

        return system_message

    def _build_user_message(self):
        """build messages"""

        # logging.info("InsightBasePrompt => _build_user_message => CALLED")

        user_message = str(self.user_message_template)
        user_message = user_message.replace("--DATA--", str(self.data)).replace(
            "--USER_DELIMITER--", self.user_delimiter
        )

        user_message = user_message.replace(
            "--INSIGHT_DELIMITER--", self.insight_delimiter
        )
        user_message = user_message.replace("--INSIGHT_DATA--", self.insight)

        return user_message

In [ ]:
class AnalyseBasePrompt(BasePrompt):
    """ """

    def __init__(
        self,
        input: str,
        output: str,
        insight: str,
        system_message_template: str = None,
        user_message_template: str = None,
        user_delimiter: str = None,
        insight_delimiter: str = None,
    ) -> None:
        """ """

        # logging.info("InsightBasePrompt ==>  __init__ CALLED")

        super().__init__(
            data=None,
            insight=insight,
            system_message_template=system_message_template,
            user_message_template=user_message_template,
            user_delimiter=user_delimiter,
            insight_delimiter=insight_delimiter,
            prompt_type="insight",
        )
        self.input = input
        self.output = output

    def _build_system_message(self):
        """build system message from template and data"""

        # logging.info("InsightBasePrompt => _build_system_message => CALLED")

        # logging.info(f"self.system_message_template : {self.system_message_template}")

        system_message = self.system_message_template.replace(
            "--USER_DELIMITER--", self.user_delimiter
        ).replace("--INSIGHT_DELIMITER--", self.insight_delimiter)

        system_message = system_message.replace("\n\n", "\n").replace("\n\n", "\n")

        return system_message

    def _build_user_message(self):
        """build messages"""

        # logging.info("InsightBasePrompt => _build_user_message => CALLED")

        user_message = str(self.user_message_template)
        user_message = (
            user_message.replace("--INPUT--", str(self.input))
            .replace("--OUTPUT--", str(self.output))
            .replace("--USER_DELIMITER--", self.user_delimiter)
        )

        user_message = user_message.replace(
            "--INSIGHT_DELIMITER--", self.insight_delimiter
        )
        user_message = user_message.replace("--INSIGHT_DATA--", self.insight)

        return user_message

### Top Level Prompts

In [ ]:
class BasicPromt(BasicBasePrompt):
    """Basic prompt for the user.

    Methods:
        - dictize() : return dict of messages
        - jsonize() : return json of messages
        - strize() : return str of messages
    """

    user_delimiter = "####"

    system_message_template = """
You are an english teacher working for people who want to correct, to fix or to improve their english language skills. \

You will be provided a text written by a non english native student. \

You will have to correct the text given by the user, taking into account the user's specific criterias. \

The user query will be delimited with \
--USER_DELIMITER-- characters.

You will answer only in english language, and you will provide a text with good tense, grammar, and vocabulary. \
    """

    user_message_template = """
--USER_DELIMITER-- 
Correct this text for me : 
--DATA--
--USER_DELIMITER-- 

    """

    def __init__(
        self,
        data: str,
        user_delimiter: str = None,
        system_message_template: str = None,
        user_message_template: str = None,
    ) -> None:
        """ """

        super().__init__(
            data=data,
            user_delimiter=user_delimiter,
            system_message_template=system_message_template,
            user_message_template=user_message_template,
        )

In [ ]:
class AnalysePrompt(AnalyseBasePrompt):
    """Insight prompt for the user.

    Methods:
        - dictize() : return dict of messages
        - jsonize() : return json of messages
        - strize() : return str of messages
    """

    user_delimiter = "####"

    insight_delimiter = "===="

    system_message_template = """
You are an english teacher working for people who want to improve their english language skills. \

You will be provided two texts : 
- the first one is a wrong text, written by a student with a low knowledge of the english language
- the second one is the correction of the first text, written by a natuve english speaker \

You will have to explain the vocabulary, the grammar, the tenses difference. You will have to give detail on what are the the differences and why they are important. 

The wrong text will be delimited with \
--USER_DELIMITER-- characters.

The context of the correction will be delimited with \
--INSIGHT_DELIMITER-- characters.

You will answer only in english language, and you will as many explainations as possible on tense, idioms, grammar, and vocabulary. \

Try to structure your answer  as much as possible: Grammar, Tense, Vocabulary.
For each explanation, give exactly 3 examples of a good implementation of the concept.

    """

    user_message_template = """
--USER_DELIMITER-- 
Explain the differences between theses two texts : 

Wrong text is : 
--INPUT--

Corrected text is : 
--OUTPUT--

--USER_DELIMITER-- 


--INSIGHT_DELIMITER--
In your correction, try to take into account the language level and the language specification of various countries : \
--INSIGHT_DATA--
--INSIGHT_DELIMITER--
    """

    def __init__(
        self,
        input: str,
        output: str,
        insight: str | list | dict,
        user_delimiter: str = None,
        insight_delimiter: str = None,
        system_message_template: str = None,
        user_message_template: str = None,
    ) -> None:
        """ """

        super().__init__(
            input=input,
            output=output,
            insight=insight,
            user_delimiter=user_delimiter,
            insight_delimiter=insight_delimiter,
            system_message_template=system_message_template,
            user_message_template=user_message_template,
        )

In [ ]:
# class Jalait:
#     """ """

#     def __init__(
#         self,
#         input_text: str,
#         lang_level: str = "Standard or daily",
#         lang_country: str = "American",
#         fix_text: bool = True,
#         grammar: bool = True,
#         audio: bool = False,
#         idioms: bool = False,
#     ) -> None:
#         """ """
#         self.input_text = input_text
#         self.lang_level = lang_level
#         self.lang_country = lang_country
#         self.fix_text = fix_text
#         self.grammar = grammar
#         self.audio = audio
#         self.idioms = idioms
#         self.output = ""
#         self.analysis = ""

#         self.config = config = manage_env()
#         self.api_key = config.get("OPENAI_API_KEY", "Error")

#     def fix(self):
#         # insight
#         insight = ""
#         if self.lang_level != "-":
#             insight += f"Use a {self.lang_level} level of language. \n"
#         if self.lang_country != "-":
#             insight += f"Use a {self.lang_country} english vocabulary/style. \n"
#         if self.idioms:
#             insight += f"Do not hesitate to use specific {self.lang_country} idioms, in accordance to the {self.lang_level} level of language expected. \n"

#         self.insight = insight

#         # prompt
#         prompt = Prompts.Insight(self.input_text, insight)

#         # completion
#         output = completion_from_messages(
#             prompt.dictize(),
#             api_key=self.api_key,
#         )

#         self.output = output

#         return output

#     def analyze(self):
#         """ """

#         prompt = Prompts.Analyze(self.input_text, self.output, self.insight)

#         # completion
#         analysis = completion_from_messages(
#             prompt.dictize(),
#             api_key=self.api_key,
#         )

#         self.analysis = analysis

#         return analysis